# Data Cleaning

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhysData/course/blob/main/docs/notebooks/cleaning_notebook.ipynb)

If you're running this in Colab, make sure to save a copy of the notebook in Google Drive to save your changes.

Here You will learn some techniques to use pandas to clean a dataset. We will use data from the Extrasolar planet encyclopedia which is structured within a csv file. It is very messy as it comes from different sources.
Our task will be to clean and filter everything to get a comprehensive dataframe. The questions we will be trying to answer are: How does the mass of the planets correlates with their radius? What does this tells us about the planets' compositions?

Check out the appendix at the end for a different way of organising the dataset to answer a different set of questions!

In [ ]:
# If you're running this notebook, uncomment the code in this cell to install the required packages.
# ! pip install pandas
# ! pip install seaborn
# ! pip install matplotlib
# ! pip install numpy

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sb
import numpy as np

## 1. Loading the csv file and looking at the uncleaned data 

In [6]:
kepler_base = pd.read_csv('C:/Users/druit/Desktop/Uni Documents/Modules year 3/Physics in Society/Project/kepler.csv')



###########STEP 1: INITIAL LOOK

#looking at the first 20 rows of data and getting the total size of the dataset
print(kepler_base.head(20))
print(f'Rows: {kepler_base.shape[0]} ; Columns: {kepler_base.shape[1]}')

                    # name planet_status   mass  mass_error_min  \
0    OGLE-2016-BLG-1469L b     Confirmed  13.60            3.00   
1                 11 Com b     Confirmed  19.40            1.50   
2                 11 Oph b     Confirmed  21.00            3.00   
3                 11 UMi b     Confirmed  10.50            2.47   
4                 14 And b     Confirmed   5.33            0.57   
5                 14 Her b     Confirmed   4.64            0.19   
6               16 Cyg B b     Confirmed   1.68            0.07   
7                 18 Del b     Confirmed  10.30             NaN   
8               1I/2017 U1     Confirmed    NaN             NaN   
9              1RXS 1609 b     Confirmed  14.00            3.00   
10          1SWASP J1407 b     Confirmed  20.00            6.00   
11                24 Sex b     Confirmed   1.99            0.38   
12                24 Sex c     Confirmed   0.86            0.22   
13       2M 0103-55 (AB) b     Confirmed  13.00            1.0

In [5]:
#getting the names of the different columns
print(list(kepler_base.columns))

['# name', 'planet_status', 'mass', 'mass_error_min', 'mass_error_max', 'mass_sini', 'mass_sini_error_min', 'mass_sini_error_max', 'radius', 'radius_error_min', 'radius_error_max', 'orbital_period', 'orbital_period_error_min', 'orbital_period_error_max', 'semi_major_axis', 'semi_major_axis_error_min', 'semi_major_axis_error_max', 'eccentricity', 'eccentricity_error_min', 'eccentricity_error_max', 'inclination', 'inclination_error_min', 'inclination_error_max', 'angular_distance', 'discovered', 'updated', 'omega', 'omega_error_min', 'omega_error_max', 'tperi', 'tperi_error_min', 'tperi_error_max', 'tconj', 'tconj_error_min', 'tconj_error_max', 'tzero_tr', 'tzero_tr_error_min', 'tzero_tr_error_max', 'tzero_tr_sec', 'tzero_tr_sec_error_min', 'tzero_tr_sec_error_max', 'lambda_angle', 'lambda_angle_error_min', 'lambda_angle_error_max', 'impact_parameter', 'impact_parameter_error_min', 'impact_parameter_error_max', 'tzero_vr', 'tzero_vr_error_min', 'tzero_vr_error_max', 'k', 'k_error_min', '

## Starting the data cleaning

Start by creating a list of columns that you want to drop.For example we don't really need the error columns for what we want to do. They might have been needed if we have different questions to resolve. For example, had one of the questions been about the link between detection methods used and the success rate we would have wanted to keep these columns as they would have provided info on the apparatus used. We will use several methods for deciding which columns to discard:

        - Some columns such as the 'error' ones are not of interest to us. We will discard them by identifying that they all have the word 'error' within their title.
        - Some columns have not got information within them (every row has the same answer or is empty)
        - Some columns have information, but not enough for it to be interesting.
        - Some columns won't be useful in answering the question and we will manually select them
        
We will use the function df.drop()

In [14]:
#list of column names in dataset
column_name = kepler_base.columns

#list of columns to drop
columns_drop = []


#first round: dropping the error columns
for col in column_name:
    if 'error' in col:
        columns_drop.append(col)

kepler_drop = kepler_base.drop(columns= columns_drop)

We notice that some rows have infinite values. This doesn't give us any more information than nan values so for ease of reading we will transform these inf values into nan ones. It will enable us to quickly indentify null cells. This will also be useful for the next steps where we want to look at the number of individual values a column has.

In [15]:
def transfer_inf(df):
    for column in df:
        infinities = (df[column] == np.inf)
        if infinities.sum() > 0:
            df.loc[infinities, column] = 'nan'

    return df

kepler_drop = transfer_inf(kepler_drop)

In [7]:
#looking at the data there seem to be some columns that do not provide any interesting information (example: planet status). 
#We can look at the different values in these columns and see whether they are all the same or not.
print(kepler_base['planet_status'].unique())

['Confirmed']


As all stars have the same status we are not getting any additional information from this column. We cannot individually look at all columns to see the number of unique values they have. The snippet of code below will select all columns that have only 1 singular value.

In [16]:
singular = []
for col in kepler_base:
    list = kepler_base[col]

    if len(list.unique()) < 2:
        #print(col)
        singular.append(col)
        #print(list.unique())

kepler_drop=kepler_drop.drop(columns= singular)

looking at the columns we see that the descriptive ones (such as the star's name) will not be of any use to us. Because they do not have any attributes in common we need to manually decide which ones we want to remove.

In [17]:
descriptivecol = ['# name', 'publication_status', 'alternate_names', 'star_name', 'star_alternate_names', 
                  'radius_detection_type', 'mass_detection_type', 'detection_type', 'log_g', 'star_sp_type', 
                  'star_detected_disc', 'star_magnetic_field','updated', 'discovered']

kepler_drop = kepler_drop.drop(columns = descriptivecol)

Some columns also seem to not have many values in them. If less than 30% of a column is filled, we can drop it as it won't give a good representation of reality. We need to find a way to calculate the number of filled values in a column. We do this by getting the percentage of nan within a column.


In [18]:
# function to drop columns based off of their percentage of nan values. Notice that here the percentage relates to the null 
# values and not the valid ones
def drop_percentage(df, perc):
    lowperc = []

    for col in df:
        na_num = df[col].isnull().sum(skipna=False)
        #print(na_num)
        colperc = (na_num/df.shape[0])*100
        if colperc > perc and col !='molecules':
            #print(f'column: {col}, percentage: {colperc}')
            lowperc.append(col)
    #print(lowperc)

    df=df.drop(columns=lowperc)
    #print(df.columns)
    return df

kepler_drop= drop_percentage(kepler_drop, 70)

We can now look at what the dataset looks like. We notice that there are now less columns. With 17 columns are dataset has become manageable whilst still having enough details to gather some interesting graphs. This is the perfect spot to save our cleaned csv file under a new name using the to_csv function. We then take our main dataframe to be this cleaned one with the usual read_csv function.

In [19]:
print(kepler_drop.head(10))
print(f'number of columns: {kepler_drop.shape[1]}')
kepler_drop.to_csv('C:/Users/druit/Desktop/Uni Documents/Modules year 3/Physics in Society/Project/kepler_cleaned.csv', index = False)

    mass    radius  orbital_period  semi_major_axis  eccentricity   molecules  \
0  13.60       NaN             NaN             0.33           NaN         NaN   
1  19.40       NaN          326.03             1.29         0.231         NaN   
2  21.00       NaN       730000.00           243.00           NaN         NaN   
3  10.50       NaN          516.22             1.54         0.080         NaN   
4   5.33       NaN          185.84             0.83         0.000         NaN   
5   4.64       NaN         1773.40             2.77         0.369         NaN   
6   1.68       NaN          799.50             1.68         0.689         NaN   
7  10.30       NaN          993.30             2.60         0.080         NaN   
8    NaN  0.000002             NaN              NaN         1.196         NaN   
9  14.00  1.700000             NaN           330.00           NaN  H2O, CO, K   

           ra        dec  mag_v  mag_j  mag_h  mag_k  star_distance  \
0  271.945750 -26.289889    NaN    Na

If you wish to get an overview of the remaining topics in this course, click the button below.

[![Back to main website](../media/back4.png)](https://physdata.vercel.app/)